In [ ]:
!pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-1dgiv3n3/gym-walk_82cc11dc9ef14457930f9270a61854f4
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-1dgiv3n3/gym-walk_82cc11dc9ef14457930f9270a61854f4
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=07c3dc7e1104f06c040542e7ae1d31a4b15a5a538ea6fd599ef6faf2387fadd5
  Stored in directory: /tmp/pip-ephem-wheel-cache-iez83gjt/wheels/60/02/77/2dd9f31df8d13bc7c014725f4002e29d0fc3ced5e8ac08e1cf
Successfully built gym-walk


In [ ]:
import warnings ; warnings.filterwarnings('ignore')

import gym
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123);

In [ ]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [ ]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [ ]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [ ]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [ ]:
init_state

0

In [ ]:
state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(state, reward, done, info))

state:0 - reward:0.0 - done:False - info:{'prob': 0.3333333333333333}


In [ ]:
pi_frozenlake = lambda s: {
    0: RIGHT,
    1: DOWN,
    2: RIGHT,
    3: LEFT,
    4: DOWN,
    5: LEFT,
    6: RIGHT,
    7:LEFT,
    8: UP,
    9: DOWN,
    10:LEFT,
    11:DOWN,
    12:RIGHT,
    13:RIGHT,
    14:DOWN,
    15:LEFT #Stop
}[s]
print_policy(pi_frozenlake, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Policy:
| 00      > | 01      v | 02      > | 03      < |
| 04      v |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [ ]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake, goal_state=goal_state) * 100,
    mean_return(env, pi_frozenlake)))

Reaches goal 10.00%. Obtains an average undiscounted return of 0.1000.


In [ ]:
# Create your own policy

pi_2 = lambda s: {
    0: UP,
    1: DOWN,
    2: LEFT,
    3: RIGHT,
    4: DOWN,
    5: LEFT,
    6: DOWN,
    7: LEFT,
    8: RIGHT,
    9: DOWN,
    10: LEFT,
    11: DOWN,
    12: RIGHT,
    13: LEFT,
    14: DOWN,
    15: UP #Stop
}[s]
print("Name: Koti Sai Sankar")
print("Register Number: 212222240111")
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)


Name: Koti Sai Sankar
Register Number: 212222240111
Policy:
| 00      ^ | 01      v | 02      < | 03      > |
| 04      v |           | 06      v |           |
| 08      > | 09      v | 10      < |           |
|           | 13      < | 14      v |           |


In [ ]:
# Find the probability of success and the mean return of you your policy

print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state) * 100,
    mean_return(env, pi_2)))

Reaches goal 4.00%. Obtains an average undiscounted return of 0.0400.


In [ ]:
# Compare your policy with the first policy
'''
pi_2 policy helps to reach the goal with 4% sucess, where undiscounted return is 0.04
pi_frozenlake helps to reach the goal with 10% sucess, where undiscounted return is 0.1
pi_frozenlake is the best policy as the sucess is 10% when compared to pi_2
'''

'\npi_2 policy helps to reach the goal with 4% sucess, where undiscounted return is 0.04\npi_frozenlake helps to reach the goal with 10% sucess, where undiscounted return is 0.1\npi_frozenlake is the best policy as the sucess is 10% when compared to pi_2\n'

In [ ]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    # Write your code here to evaluate the given policy
    while True:
      V=np.zeros(len(P))
      for s in range(len(P)):
        for prob,next_state,reward,done in P[s][pi(s)]:
           V[s]+=prob*(reward+gamma *prev_V[next_state]*(not done))
      if np.max(np.abs(prev_V-V))<theta:
        break
      prev_V=V.copy()
    return V

In [ ]:
# Code to evaluate the first policy
V1 = policy_evaluation(pi_frozenlake, P,gamma=0.99)
print_state_value_function(V1, P, n_cols=4, prec=5)

State-value function:
| 00 0.11448 | 01 0.08191 | 02 0.13372 | 03 0.06586 |
| 04 0.15053 |           | 06 0.20562 |           |
| 08 0.30562 | 09 0.46997 | 10 0.48938 |           |
|           | 13 0.62915 | 14 0.80739 |           |


In [ ]:
# Code to evaluate the second policy
# Write your code here
V2 = policy_evaluation(pi_2, P,gamma=0.99)
print_state_value_function(V2, P, n_cols=4, prec=5)

State-value function:
| 00 0.02362 | 01 0.02433 | 02 0.05012 | 03    0.0 |
| 04 0.02124 |           | 06 0.07742 |           |
| 08 0.04312 | 09 0.10944 | 10 0.2346 |           |
|           | 13 0.0539 | 14 0.52406 |           |


In [ ]:
# Comparing the two policies
# Compare the two policies based on the value function using the above equation and find the best policy

# The first policy produces higher values across all states compared to the second policy.
# Since the first policy has consistently higher state-value function values than the second policy, the first policy is better

In [ ]:
V1

array([0.11448243, 0.08190533, 0.13371554, 0.06585989, 0.15052869,
       0.        , 0.20562317, 0.        , 0.30561885, 0.46997019,
       0.48938498, 0.        , 0.        , 0.62914826, 0.80739143,
       0.        ])

In [ ]:
V2

array([0.02361582, 0.02433145, 0.05011584, 0.        , 0.02123992,
       0.        , 0.0774189 , 0.        , 0.04312348, 0.10943729,
       0.23460273, 0.        , 0.        , 0.05390195, 0.52406116,
       0.        ])

In [ ]:
V1>=V2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [ ]:
if(np.sum(V1>=V2)==16):
  print("The first policy is the better policy")
elif(np.sum(V2>=V1)==16):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")

The first policy is the better policy
